In [1]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QDialog,
    QPushButton, QVBoxLayout, QHBoxLayout, QMessageBox,
    QLabel, QCheckBox, QDialogButtonBox
)


class CategoryDialog(QDialog):
    """카테고리 선택 창"""
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("카테고리 선택")
        self.resize(300, 200)

        layout = QVBoxLayout()

        # 예시 체크박스
        self.case1 = QCheckBox("Case 1")
        self.case2 = QCheckBox("Case 2")
        self.case3 = QCheckBox("Case 3")

        layout.addWidget(self.case1)
        layout.addWidget(self.case2)
        layout.addWidget(self.case3)

        # 확인 / 취소 버튼
        btn_box = QDialogButtonBox(QDialogButtonBox.Ok | QDialogButtonBox.Cancel)
        btn_box.accepted.connect(self.accept_clicked)
        btn_box.rejected.connect(self.reject)
        layout.addWidget(btn_box)

        self.setLayout(layout)

    def accept_clicked(self):
        # 예시 로직: 카테고리 선택 결과 확인
        checked_list = []
        if self.case1.isChecked():
            checked_list.append("Case 1")
        if self.case2.isChecked():
            checked_list.append("Case 2")
        if self.case3.isChecked():
            checked_list.append("Case 3")

        QMessageBox.information(self, "카테고리 확정", f"선택된 카테고리: {checked_list}")
        self.accept()


class MergeDialog(QDialog):
    """Merge 옵션 창"""
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Merge 옵션")
        self.resize(300, 220)

        layout = QVBoxLayout()

        # 예시 체크박스
        self.chk_dup = QCheckBox("중복 제거")
        self.chk_file = QCheckBox("파일 합치기")
        self.chk_outchk = QCheckBox("출력폴더 체크")
        self.chk_outpath = QCheckBox("Output 경로")

        layout.addWidget(self.chk_dup)
        layout.addWidget(self.chk_file)
        layout.addWidget(self.chk_outchk)
        layout.addWidget(self.chk_outpath)

        # 확인 버튼
        btn_box = QDialogButtonBox(QDialogButtonBox.Ok | QDialogButtonBox.Cancel)
        btn_box.accepted.connect(self.on_accept)
        btn_box.rejected.connect(self.reject)
        layout.addWidget(btn_box)

        self.setLayout(layout)

    def on_accept(self):
        # 예시 로직: 체크박스 상태 확인
        selections = []
        if self.chk_dup.isChecked():
            selections.append("중복 제거")
        if self.chk_file.isChecked():
            selections.append("파일 합치기")
        if self.chk_outchk.isChecked():
            selections.append("출력폴더 체크")
        if self.chk_outpath.isChecked():
            selections.append("Output 경로")

        QMessageBox.information(self, "Merge 옵션", f"설정된 옵션: {selections}")
        self.accept()


class QuestionDialog(QDialog):
    """문제 분리 창"""
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("문제 분리")
        self.resize(300, 180)

        layout = QVBoxLayout()
        label = QLabel("문제 분리 옵션 예시")
        layout.addWidget(label)

        btn = QPushButton("분리 실행")
        btn.clicked.connect(self.execute_split)
        layout.addWidget(btn)

        self.setLayout(layout)

    def execute_split(self):
        # 예시 동작
        QMessageBox.information(self, "문제 분리", "문제 분리 실행됨")


class AnswerDialog(QDialog):
    """정답 분리 창"""
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("정답 분리")
        self.resize(300, 180)

        layout = QVBoxLayout()
        label = QLabel("정답 분리 옵션 예시")
        layout.addWidget(label)

        btn = QPushButton("분리 실행")
        btn.clicked.connect(self.execute_split)
        layout.addWidget(btn)

        self.setLayout(layout)

    def execute_split(self):
        # 예시 동작
        QMessageBox.information(self, "정답 분리", "정답 분리 실행됨")


class CSVDialog(QDialog):
    """CSV 파일로 창"""
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("CSV 파일로")
        self.resize(300, 180)

        layout = QVBoxLayout()
        label = QLabel("CSV 저장 옵션 예시")
        layout.addWidget(label)

        btn = QPushButton("저장 실행")
        btn.clicked.connect(self.save_csv)
        layout.addWidget(btn)

        self.setLayout(layout)

    def save_csv(self):
        # 예시 동작
        QMessageBox.information(self, "CSV 저장", "CSV로 저장 실행됨")


class MainWindow(QMainWindow):
    """메인 창"""
    def __init__(self):
        super().__init__()
        self.setWindowTitle("메인 GUI")
        self.resize(500, 300)

        # 중앙 위젯 (전체 레이아웃을 담을 컨테이너)
        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        # 메인 레이아웃(수직)
        main_layout = QVBoxLayout(central_widget)

        # 상단: 왼쪽과 오른쪽에 버튼 2개씩 배치 (QHBoxLayout 안에 QVBoxLayout 2개)
        top_layout = QHBoxLayout()

        # 왼쪽 버튼 2개
        left_layout = QVBoxLayout()
        self.btn_category = QPushButton("카테고리 선택")
        self.btn_merge = QPushButton("Merge 옵션")
        left_layout.addWidget(self.btn_category)
        left_layout.addWidget(self.btn_merge)

        # 오른쪽 버튼 2개
        right_layout = QVBoxLayout()
        self.btn_q_split = QPushButton("문제 분리")
        self.btn_a_split = QPushButton("정답 분리")
        right_layout.addWidget(self.btn_q_split)
        right_layout.addWidget(self.btn_a_split)

        # top_layout에 왼쪽/오른쪽 레이아웃 추가
        top_layout.addLayout(left_layout)
        top_layout.addSpacing(50)  # 왼쪽, 오른쪽 간격 조절(필요시)
        top_layout.addLayout(right_layout)

        # 하단: CSV 버튼 가운데 배치
        bottom_layout = QHBoxLayout()
        self.btn_csv = QPushButton("CSV 파일로")
        bottom_layout.addStretch()
        bottom_layout.addWidget(self.btn_csv)
        bottom_layout.addStretch()

        # 메인 레이아웃에 top_layout, bottom_layout 순서대로 추가
        main_layout.addLayout(top_layout)
        main_layout.addStretch()    # 위/아래 공간 확장
        main_layout.addLayout(bottom_layout)

        # 버튼 클릭 시 다이얼로그 열기
        self.btn_category.clicked.connect(self.open_category_dialog)
        self.btn_merge.clicked.connect(self.open_merge_dialog)
        self.btn_q_split.clicked.connect(self.open_question_dialog)
        self.btn_a_split.clicked.connect(self.open_answer_dialog)
        self.btn_csv.clicked.connect(self.open_csv_dialog)

    def open_category_dialog(self):
        dialog = CategoryDialog(self)
        dialog.exec_()

    def open_merge_dialog(self):
        dialog = MergeDialog(self)
        dialog.exec_()

    def open_question_dialog(self):
        dialog = QuestionDialog(self)
        dialog.exec_()

    def open_answer_dialog(self):
        dialog = AnswerDialog(self)
        dialog.exec_()

    def open_csv_dialog(self):
        dialog = CSVDialog(self)
        dialog.exec_()
        

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33368\3300881767.py:9: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class CategoryDialog(QDialog):
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33368\3300881767.py:49: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class MergeDialog(QDialog):
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33368\3300881767.py:93: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class QuestionDialog(QDialog):
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33368\3300881767.py:115: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class AnswerDialog(QDialog):
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33368\3300881767.py:137: DeprecationWarning: sipPyTypeDic

In [1]:
import sys
import csv
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QDialog,
    QPushButton, QVBoxLayout, QHBoxLayout, QMessageBox,
    QLabel, QCheckBox, QDialogButtonBox
)
from PyQt5.QtCore import Qt


CSV_PATH = "./csv_file/categories.csv"


class CategoryDialog(QDialog):
    """
    1) CSV 파일(categories.csv)을 읽어
       - year, difficulty, subject의 유니크 목록을 구한다.
    2) 각 목록을 체크박스로 표시한다.
    3) OK 버튼 클릭 시,
       - 선택된 체크박스들의 조건에 맞는 row를 필터링
       - 해당 row들의 file_name을 수집하여 결과로 보여준다.
    """

    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("카테고리 선택")
        self.resize(400, 300)

        # CSV 읽어서 데이터 보관 (필요하면 pandas 사용 가능)
        self.csv_data = self.load_csv_data(CSV_PATH)

        # year, difficulty, subject 컬럼의 유니크 값 추출
        unique_years = sorted(set(row["year"] for row in self.csv_data))
        unique_diffs = sorted(set(row["difficulty"] for row in self.csv_data))
        unique_subjects = sorted(set(row["subject"] for row in self.csv_data))

        # 레이아웃 구성
        main_layout = QVBoxLayout(self)

        info_label = QLabel("원하는 카테고리를 선택하세요.")
        main_layout.addWidget(info_label)

        # 상단 레이아웃(세로 3등분: year / difficulty / subject)
        top_layout = QHBoxLayout()
        main_layout.addLayout(top_layout)

        # 1) Year 체크박스 세트
        year_layout = QVBoxLayout()
        year_layout.addWidget(QLabel("<b>Year</b>"))
        self.year_checkboxes = []
        for y in unique_years:
            cb = QCheckBox(str(y))
            year_layout.addWidget(cb)
            self.year_checkboxes.append(cb)

        # 2) Difficulty 체크박스 세트
        diff_layout = QVBoxLayout()
        diff_layout.addWidget(QLabel("<b>Difficulty</b>"))
        self.diff_checkboxes = []
        for d in unique_diffs:
            cb = QCheckBox(d)
            diff_layout.addWidget(cb)
            self.diff_checkboxes.append(cb)

        # 3) Subject 체크박스 세트
        subj_layout = QVBoxLayout()
        subj_layout.addWidget(QLabel("<b>Subject</b>"))
        self.subj_checkboxes = []
        for s in unique_subjects:
            cb = QCheckBox(s)
            subj_layout.addWidget(cb)
            self.subj_checkboxes.append(cb)

        top_layout.addLayout(year_layout)
        top_layout.addStretch()
        top_layout.addLayout(diff_layout)
        top_layout.addStretch()
        top_layout.addLayout(subj_layout)

        # 확인 / 취소 버튼
        btn_box = QDialogButtonBox(QDialogButtonBox.Ok | QDialogButtonBox.Cancel, parent=self)
        btn_box.accepted.connect(self.on_accept)
        btn_box.rejected.connect(self.reject)

        main_layout.addWidget(btn_box)

    def load_csv_data(self, filepath):
        """CSV를 읽어 row 리스트 반환 (딕셔너리 형태)"""
        data = []
        with open(filepath, newline='', encoding='utf-8') as f:
            # reader = csv.DictReader(f, fieldnames=["file_name", "year", "difficulty", "subject"])
            reader = csv.DictReader(f)
            # DictReader 사용 시, 기본적으로 첫 줄이 헤더로 인식됨
            # 만약 CSV에 헤더가 있는 형태라면 fieldnames 설정을 빼고 사용하시거나
            # 첫 줄을 건너뛰는 식으로 처리해 주세요.
            for row in reader:
                # 필요에 따라 문자열 trim, lower 등 처리 가능
                data.append({
                    "file_name": row["file_name"],
                    "year": row["year"],
                    "difficulty": row["difficulty"],
                    "subject": row["subject"]
                })
        return data

    def on_accept(self):
        """확인 버튼 클릭 시, 체크된 카테고리에 맞춰 필터링"""
        selected_years = []
        for cb in self.year_checkboxes:
            if cb.isChecked():
                selected_years.append(cb.text())

        selected_diffs = []
        for cb in self.diff_checkboxes:
            if cb.isChecked():
                selected_diffs.append(cb.text())

        selected_subjs = []
        for cb in self.subj_checkboxes:
            if cb.isChecked():
                selected_subjs.append(cb.text())

        # 선택된 값이 없다면 전체로 간주할 수도 있고,
        # 또는 "하나 이상 체크해주세요"라는 메시지를 띄울 수도 있습니다.
        # 여기서는 "체크 안 한 카테고리는 무시" 로 처리하기 위해, 
        # 하나도 없으면 '모든 값 허용'으로 해석하도록 하겠습니다.
        
        def match_value(val, selected_list):
            if not selected_list:  # 아무것도 안 고르면 True 처리
                return True
            return val in selected_list

        matched_files = []
        for row in self.csv_data:
            if (match_value(row["year"], selected_years) and
                match_value(row["difficulty"], selected_diffs) and
                match_value(row["subject"], selected_subjs)):
                matched_files.append(row["file_name"])

        if matched_files:
            QMessageBox.information(self, "결과", 
                f"선택된 카테고리에 해당하는 파일:\n\n{matched_files}")
        else:
            QMessageBox.warning(self, "결과", "해당 조건에 맞는 파일이 없습니다.")

        # 다이얼로그 닫기
        self.accept()


class MainWindow(QMainWindow):
    """메인 창"""
    def __init__(self):
        super().__init__()
        self.setWindowTitle("메인 GUI")
        self.resize(500, 300)

        # 중앙 위젯 (전체 레이아웃을 담을 컨테이너)
        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        # 메인 레이아웃(수직)
        main_layout = QVBoxLayout(central_widget)

        self.btn_category = QPushButton("카테고리 선택")
        main_layout.addWidget(self.btn_category)

        # 다른 버튼들(Merge, 문제 분리, 정답 분리, CSV 등)은 필요 시 추가
        # 여기서는 예시로 카테고리 버튼만 배치

        # 버튼 클릭 시 다이얼로그 열기
        self.btn_category.clicked.connect(self.open_category_dialog)

    def open_category_dialog(self):
        dialog = CategoryDialog(self)
        dialog.exec_()


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_27936\1001520137.py:14: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class CategoryDialog(QDialog):
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_27936\1001520137.py:150: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class MainWindow(QMainWindow):


SystemExit: 0

c:\Users\CHY\.conda\envs\msword\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
app = QApplication(sys.argv)
window = MainWindow()
window.show()


# sys.exit(app.exec_()) # .py에서 사용
app.exec_() # .ipynb에서 사용용

0